In [ ]:
#import libraries
import pandas as pd
import numpy as np

In [ ]:
#Read tsv file
dataFrame = pd.read_csv("version1/reducedusersrows.tsv", sep = "\t", header=0, encoding = "latin1")

In [ ]:
#Read tsv file
result = pd.read_csv("version1/allsinglerows.tsv", sep = "\t", header=0, encoding = "latin1")

In [ ]:
#Retreive user id and events columns
eventListRows = dataFrame[["post_uniqueId","post_event_list"]].reset_index(drop=True)

In [ ]:
#converting these columns into string type
eventListRows['post_uniqueId'] = eventListRows['post_uniqueId'].astype(str)
eventListRows['post_event_list'] = eventListRows['post_event_list'].astype(str)

In [ ]:
#Sort the rows by user id
eventListRowsSort = eventListRows.sort_values(by=['post_uniqueId'])
eventListRowsSort = eventListRowsSort.reset_index(drop=True)

In [ ]:
#Merge all the events performed by user into single row
eventListRowsGroup = eventListRowsSort.groupby(['post_uniqueId'])['post_event_list'].apply(lambda x: ','.join(x)).reset_index()

In [ ]:
colNames = ['country','post_browser_height','post_browser_width','timespent','cartcount','eventcount']

In [ ]:
for c in colNames:
    eventListRowsGroup[c] = result[c]

In [ ]:
#create an empty list
eventList=[]

In [ ]:
#loop through each cell of event list column and remove duplicate events
for index, row in eventListRowsGroup.iterrows():
    templist = set(pd.Series(eventListRowsGroup['post_event_list'].iloc[index].split(',')).tolist())
    eventList.append(",".join(templist))
    #print(index)

In [ ]:
eventList

In [ ]:
#Assigning events which doesnt contain duplicated events
eventListRowsGroup['post_event_list'] = eventList

In [ ]:
#import Counter from library
from collections import Counter

In [ ]:
#Get count of events
eventsCount  = pd.Series(eventListRowsGroup['post_event_list'].str.replace('[\[\]\']','').str.split(',').map(Counter).sum())

In [ ]:
#Initialize threshold
thre1 = 4034/2 #50%
thre2 = 4034/4 #25%
#Filter using the threshold. All the values less than thre will be substitued witn nan values
eventsCountThre = eventsCount.where(eventsCount<thre1)

In [ ]:
#Filtering the events which have ben performed atleast by 25% of users and atmost 50%
eventsCountThre = eventsCountThre.where(eventsCount>thre2)

In [ ]:
#Remove nan values
eventsCountThreWithoutNull = eventsCountThre.dropna()

In [ ]:
#Get all unique events after thre
eventNumbersList = eventsCountThreWithoutNull.index.tolist()

In [ ]:
#Count of events
len(eventNumbersList)

In [ ]:
#Add the events as columns to dataframe
for i in eventNumbersList:
    name = 'event'+i
    eventListRowsGroup[name] = 0

In [ ]:
#eventListRowsGroup

In [ ]:
for index, row in eventListRowsGroup.iterrows():
    templist = pd.Series(eventListRowsGroup['post_event_list'].iloc[index].replace('[\[\]\']','').split(',')).tolist()
    print(index)
    commonlist = [value for value in eventNumbersList if value in templist]
    for event in commonlist:
        eventname = 'event'+event
        eventListRowsGroup[eventname].iloc[index] = 1

In [ ]:
#eventListRowsGroup

In [ ]:
eventListRowsGroup['purchased'] = result['purchased']

In [ ]:
#eventListRowsGroup

In [ ]:
eventListRowsGroup

In [ ]:
#Save dataframe to a tsv file
eventListRowsGroup.to_csv("version1/modeldata.tsv", sep="\t", index=False)